# Peak and plot simultaneously

In [203]:
import csv
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
from IPython.display import Image
from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy
import os, os.path

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys
# search path for modules
# look @ https://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
import remote_sensing_core as rc
import remote_sensing_core as rcp

start = time.time()

# Directories

In [204]:
data_dir = "/Users/hn/Documents/01_research_data/Ag_check_point/" + \
           "remote_sensing/01_NDVI_TS/Grant/No_EVI/Grant_10_cloud/Grant_2017/"

In [205]:
output_dir = data_dir
plot_dir_base = data_dir + "plots/"

# Data Reading

In [206]:
file_names = ["Grant_2017_TS.csv"]
file_N = file_names[0]
a_df = pd.read_csv(data_dir + file_N)

# The following columns do not exist in the old data
a_df['CovrCrp'] = "NA" 
a_df['DataSrc'] = "NA" 

a_df = rc.initial_clean(a_df)
a_df.head(2)

,Acres,B2,B3,B4,B8,CropGrp,CropTyp,ExctAcr,IntlSrD,Irrigtn,...,Shap_Ar,Shp_Lng,Source,TRS,county,doy,year,geo,CovrCrp,DataSrc
0,36,0.092407,0.070650,0.071444,0.109662,Herb,Mint,35.813572,2003/07/01 00:00:00,Drip,...,144932.383795,1572.482519,WSDA,T16R27E23,Grant,62.0,2017.0,"{""type"":""Polygon"",""coordinates"":[[[-119.404844...",NA,NA
1,36,0.105605,0.107085,0.075052,0.436749,Herb,Mint,35.813572,2003/07/01 00:00:00,Drip,...,144932.383795,1572.482519,WSDA,T16R27E23,Grant,119.0,2017.0,"{""type"":""Polygon"",""coordinates"":[[[-119.404844...",NA,NA


In [170]:
an_EE_TS = a_df.copy()
an_EE_TS = an_EE_TS.iloc[1:1000]

an_EE_TS = rc.initial_clean(an_EE_TS)

### List of unique polygons
polygon_list = an_EE_TS['geo'].unique()
print(len(polygon_list))

output_columns = ['Acres', 'CovrCrp', 'CropGrp', 'CropTyp',
                  'DataSrc', 'ExctAcr', 'IntlSrD', 'Irrigtn', 'LstSrvD', 'Notes',
                  'RtCrpTy', 'Shap_Ar', 'Shp_Lng', 'TRS', 'county', 'year', 'geo',
                  'peak_Doy', 'peak_value']

all_polygons_and_their_peaks = pd.DataFrame(data=None, 
                                            index=np.arange(3*len(an_EE_TS)), 
                                            columns=output_columns)

double_columns = ['Acres', 'CovrCrp', 'CropGrp', 'CropTyp',
                  'DataSrc', 'ExctAcr', 'IntlSrD', 'Irrigtn', 'LstSrvD', 'Notes',
                  'RtCrpTy', 'Shap_Ar', 'Shp_Lng', 'TRS', 'county', 'year', 'geo']

double_polygons = pd.DataFrame(data=None, 
                               index=np.arange(2*len(an_EE_TS)), 
                               columns=double_columns)

32


In [198]:
pointer = 0
double_pointer = 0
counter = 0
for a_poly in polygon_list:
    if (counter%1000 == 0):
        print (counter)
    counter += 1
    curr_field = an_EE_TS[an_EE_TS['geo']==a_poly]

    year = int(curr_field['year'].unique())
    plant = curr_field['CropTyp'].unique()[0]
    plant = plant.replace("/", ",")
    county = curr_field['county'].unique()[0]
    TRS = curr_field['TRS'].unique()[0]

    ### 
    ###  There is a chance that a polygon is repeated twice?
    ###

    X = curr_field['doy']
    y = curr_field['NDVI']
    freedom_df = 7
    #############################################
    ###
    ###             Smoothen
    ###
    #############################################

    # Generate spline basis with "freedom_df" degrees of freedom
    x_basis = cr(X, df=freedom_df, constraints='center')

    # Fit model to the data
    model = LinearRegression().fit(x_basis, y)

    # Get estimates
    y_hat = model.predict(x_basis)


    #############################################
    ###
    ###             find peaks
    ###
    #############################################
    # peaks_LWLS_1 = peakdetect(LWLS_1[:, 1], lookahead = 10, delta=0)
    # max_peaks = peaks_LWLS_1[0]
    # peaks_LWLS_1 = form_xs_ys_from_peakdetect(max_peak_list = max_peaks, doy_vect=X)

    peaks_spline = rc.peakdetect(y_hat, lookahead = 10, delta=0)
    max_peaks =  peaks_spline[0]
    peaks_spline = rc.form_xs_ys_from_peakdetect(max_peak_list = max_peaks, doy_vect=X)
    # print(peaks_spline)
    DoYs_series = pd.Series(peaks_spline[0])
    peaks_series = pd.Series(peaks_spline[1])

    peak_df = pd.DataFrame({ 
                       'peak_Doy': DoYs_series,
                       'peak_value': peaks_series
                      }) 


    WSDA_df = rc.keep_WSDA_columns(curr_field)
    WSDA_df = WSDA_df.drop_duplicates()
    
    if (len(peak_df)>0):
        WSDA_df = pd.concat([WSDA_df]*peak_df.shape[0]).reset_index()
        # WSDA_df = pd.concat([WSDA_df, peak_df], axis=1, ignore_index=True)
        WSDA_df = WSDA_df.join(peak_df)
        if ("index" in WSDA_df.columns):
            WSDA_df = WSDA_df.drop(columns=['index'])

        # all_polygons_and_their_peaks = all_polygons_and_their_peaks.append(WSDA_df, sort=False)

        """
        copy the .values. Otherwise the index inconsistency between
        WSDA_df and all_poly... will prevent the copying.
        """
        all_polygons_and_their_peaks.iloc[pointer:(pointer + len(WSDA_df))] = WSDA_df.values
        #
        #  if we have double peaks add them to the double_polygons
        #
        if (len(WSDA_df) == 2):
            print(plant, county, year, counter)
            WSDA_df = WSDA_df.drop(columns=['peak_Doy', 'peak_value'])
            WSDA_df = WSDA_df.drop_duplicates()
            double_polygons.iloc[double_pointer:(double_pointer + len(WSDA_df))] = WSDA_df.values
            double_pointer += len(WSDA_df)

        pointer += len(WSDA_df)
        
        #############################################
        ###
        ###             plot
        ###
        #############################################
        sub_out = "/" + plant + "/"
        plot_path = plot_dir_base + sub_out
        os.makedirs(plot_path, exist_ok=True)
        if (len(os.listdir(plot_path))<100):
            plot_title = county + ", " + plant + ", " + str(year) + " (" + TRS + ")"
            sb.set();
            fig, ax = plt.subplots(figsize=(8,6));
            ax.plot(X, y, label="NDVI data");
            ax.plot(X, y_hat, 'r', label="smoothing spline result")
            ax.scatter(DoYs_series, peaks_series, s=100, c='g', marker='*');
            ax.set_title(plot_title);
            ax.set(xlabel='DoY', ylabel='NDVI')
            ax.legend(loc="best");

            fig_name = plot_path + county + "_" + plant + "_" + str(year) + "_" + str(counter) + '.png'
            plt.savefig(fname = fig_name, \
                        dpi=500, 
                        bbox_inches='tight')
            del(plot_path, sub_out, county, plant, year)
            plt.close()

        # to make sure the reference by address thing 
        # will not cause any problem.
    del(WSDA_df)

all_polygons_and_their_peaks = all_polygons_and_their_peaks[0:(pointer+1)]
double_polygons = double_polygons[0:(double_pointer+1)]

out_name = output_dir + 'all_polygons_and_their_peaks.csv'
all_polygons_and_their_peaks.to_csv(out_name, index = False)

out_name = output_dir + 'double_polygons.csv'
double_polygons.to_csv(out_name, index = False)

0
Apple Grant 2017 28


In [199]:
plot_dir_base

'/Users/hn/Documents/01_research_data/Ag_check_point/remote_sensing/01_NDVI_TS/Grant/No_EVI/Grant_10_cloud/Grant_2017/plots/'

In [200]:
sub_out

'/Apple/'

In [201]:
plot_path = plot_dir_base + sub_out

In [202]:
plot_path

'/Users/hn/Documents/01_research_data/Ag_check_point/remote_sensing/01_NDVI_TS/Grant/No_EVI/Grant_10_cloud/Grant_2017/plots//Apple/'